# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder. Additionally, Make sure to delete or comment out any code you add to this notebook for testing purposes prior to submitting the notebook to Gradescope. Failure to do so may crash the autograder. 
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickup_locations()`

`exclude_no_trip_distance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You should not use - and do not need - any helper functions; implement the required functions by completing the function skeletons below. All of the fuctions you implement should be self-contained; each function should carry all of the code it needs to execute on its expected input(s) within its body. 

### Kernel Selection

Be sure that you select the PySpark kernel from the kernel options above, if it is not already selected. Do not use the Python3 kernel. The notebook should default to the PySpark kernel. 

### GCP and Gradescope
For submitting to Gradescope, you should use spark dataframes and dataframe operations _only_. 
You should not use SQL. 

### SparkSession Objects and SparkContexts
Note that you have access to a SparkSession object in the notebook environment in virtue of using the PySpark kernel. It can be referenced using the variable name _spark_ and does not need to be explicitly created or defined. This beavior can be leveraged to read in the initial dataset. 

You should not need to invoke _spark_ in any of your functions but the load_data() function. Do not hardcode references to _spark_ into any of your functions except for the load_data() function. 

### Assumptions Regarding Function Inputs
This question assumes that you're carrying the results of your prior work foward at each stage of data preparation and analysis. Thus, the _input_ to each of the _filtering_ functions below should be the _output_ of the _prior_ function. The inputs to the two analytical functions should be the output of the last filtering function. 

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark
import decimal

In [2]:
### Student Section - Please compete all the functions below

#### Function to return GT Username

In [3]:
#export
def user() -> str:
    """
    :return: string
    your GTUsername, NOT your 9-Digit GTId  
    """         
    return 'amehroke3'

#### 4.2 - Function to load data

In [4]:
#export
def load_data(gcp_storage_path: str) -> pyspark.sql.DataFrame:
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP  #
    # storage bucket                                               #      
    ################################################################
    spark = SparkSession.builder.appName("Load GCP Data").getOrCreate()

    # Load the data from GCS bucket
    df = spark.read.csv(gcp_storage_path, header=True, inferSchema=True)

    return df

#### 4.3 - Function to exclude trips that don't have a pickup location

In [5]:
#export
def exclude_no_pickup_locations(df: pyspark.sql.DataFrame) -> pyspark.sql.DataFrame:
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to exclude trips with no pickup locations               #
    # Note: Exclude nulls and zeros                                #        
    ################################################################
    
    filtered_df = df.filter((df["pulocationid"].isNotNull()) & (df["pulocationid"] > 0))
    
    return filtered_df


#### 4.4 - Function to exclude trips with no distance

In [6]:
#export
def exclude_no_trip_distance(df: pyspark.sql.DataFrame) -> pyspark.sql.DataFrame:
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to exclude trips with no trip distances                 #
    # Note: Exclude nulls and zeros                                #        
    ################################################################
    
    df = df.withColumn("trip_distance", df["trip_distance"].cast("decimal(38,10)"))
    
    df_filtered = df.filter((df['trip_distance'].isNotNull()) & (df['trip_distance'] > 0))
    
    
    return df_filtered

#### 4.5 - Function to include fare amount between the range of 20 to 60 Dollars

In [7]:
#export
def include_fare_range(df: pyspark.sql.DataFrame) -> pyspark.sql.DataFrame:
    
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars (including 20 and 60 dollars)               #    
    ################################################################
    
    df = df.withColumn("fare_amount", df["fare_amount"].cast("decimal(38,10)"))

    filtered_df = df.filter((df["fare_amount"] >= 20) & (df["fare_amount"] <= 60))
    
    return filtered_df

#### 4.6 - Function to get the highest tip amount

In [8]:
#export
def get_highest_tip(df: pyspark.sql.DataFrame) -> decimal.Decimal:
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get the highest tip amount                           #
    #                                                              #        
    ################################################################
    
    df = df.withColumn("tip_amount", df["tip_amount"].cast("decimal(38,10)"))

    highest_tip = df.agg({"tip_amount": "max"}).collect()[0][0]

    tip = decimal.Decimal(highest_tip).quantize(decimal.Decimal('0.01'))
    
    return tip


#### 4.7 - Function to get total toll amount

In [9]:
#export
def get_total_toll(df: pyspark.sql.DataFrame) -> decimal.Decimal:
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    
    df = df.withColumn("tolls_amount", df["tolls_amount"].cast("decimal(38,10)"))

    total_toll = df.agg({"tolls_amount": "sum"}).collect()[0][0]
    
    total = decimal.Decimal(total_toll).quantize(decimal.Decimal('0.01'))
    
    return total

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata09-08-2021.csv

In [10]:
# gcp_storage_path = "gs://amehroke3/yellow_tripdata09-08-2021.csv"
# df = load_data(gcp_storage_path)
# df.printSchema()

24/10/22 22:23:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- vendorid: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pulocationid: integer (nullable = true)
 |-- dolocationid: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



#### Print total numbers of rows in the dataframe

In [11]:
# df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [12]:
# df_no_pickup_locations = exclude_no_pickup_locations(df)
# df_no_pickup_locations.count()

3833896

#### Print total number of rows in the dataframe after exclude trips with no distance

In [13]:
# df_no_trip_distance = exclude_no_trip_distance(df_no_pickup_locations)
# df_no_trip_distance.count()

2030274

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [14]:
# df_include_fare_range = include_fare_range(df_no_trip_distance)
# df_include_fare_range.count()

255321

#### Print the highest tip amount

In [15]:
# max_tip = get_highest_tip(df_include_fare_range)
# print(max_tip)

212.00


#### Print the total toll amount

In [16]:
# total_toll = get_total_toll(df_include_fare_range)
# print(total_toll)

561387.86


24/10/22 22:25:55 ERROR TransportClient: Failed to send RPC RPC 4972505745125115194 to /10.128.0.2:35012: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException: null
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source) ~[netty-transport-4.1.77.Final.jar:4.1.77.Final]
24/10/22 22:25:55 WARN BlockManagerMasterEndpoint: Error trying to remove broadcast 21 from block manager BlockManagerId(3, amehroke3-w-1.us-central1-f.c.cse6242-439421.internal, 37059, None)
java.io.IOException: Failed to send RPC RPC 4972505745125115194 to /10.128.0.2:35012: io.netty.channel.StacklessClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:392) ~[spark-network-common_2.12-3.3.2.jar:3.3.2]
	at org.apache.spark.network.client.TransportClient$StdChannelListener.operationComplete(TransportClient.java:369) ~[spark-network-common_2.12-3.3.2.jar:3.3.2]
